In [1]:
!curl -o mesh2022.nt https://nlmpubs.nlm.nih.gov/projects/mesh/rdf/2022/mesh2022.nt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1874M  100 1874M    0     0  38.4M      0  0:00:48  0:00:48 --:--:-- 33.3M8M62 1167M    0     0  38.4M      0  0:00:48  0:00:30  0:00:18 38.6M


In [2]:
!aws s3 cp mesh2022.nt s3://osedemah-mesh-rdf/mesh2022.nt

upload: ./mesh2022.nt to s3://osedemah-mesh-rdf/mesh2022.nt        


In [7]:
!curl -X POST \
    -H 'Content-Type: application/json' \
    https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/loader -d '\
    {\
      "source" : "s3://osedemah-mesh-rdf/mesh2022.nt",\
      "format" : "ntriples",\
      "iamRoleArn" : "arn:aws:iam::812630329048:role/NeptuneS3",\
      "region" : "us-west-2",\
      "failOnError" : "FALSE",\
      "parallelism" : "MEDIUM",\
      "updateSingleCardinalityProperties" : "FALSE",\
      "queueRequest" : "TRUE"\
    }'

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "003ba86c-a1fa-418f-9da1-3dbe677f986f"
    }
}

In [20]:
!curl https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/loader/003ba86c-a1fa-418f-9da1-3dbe677f986f

{
    "status" : "200 OK",
    "payload" : {
        "feedCount" : [
            {
                "LOAD_COMPLETED" : 1
            }
        ],
        "overallStatus" : {
            "fullUri" : "s3://osedemah-mesh-rdf/mesh2022.nt",
            "runNumber" : 1,
            "retryNumber" : 0,
            "status" : "LOAD_COMPLETED",
            "totalTimeSpent" : 402,
            "startTime" : 1694210786,
            "totalRecords" : 15163366,
            "totalDuplicates" : 0,
            "parsingErrors" : 0,
            "datatypeMismatchErrors" : 0,
            "insertErrors" : 0
        }
    }
}

In [1]:
from rdflib import Graph, Namespace
from rdflib.plugins.stores import sparqlstore


neptune_endpoint = "https://database-1.cluster-cclypxvaeodz.us-west-2.neptune.amazonaws.com:8182/sparql"
store = sparqlstore.SPARQLStore(neptune_endpoint, method='POST')
g = Graph(store)

MESHV = Namespace("http://id.nlm.nih.gov/mesh/vocab#")
MESH = Namespace("http://id.nlm.nih.gov/mesh/2022/")
g.bind("meshv", MESHV)
g.bind("mesh", MESH)


def query(q):
    return g.query(q)

In [2]:
descriptors = query(
    """
    SELECT ?d
    WHERE {
        {?d a meshv:TopicalDescriptor .}
        UNION
        {?d a meshv:Descriptor .}
    }
    """
)

descriptors = [str(d[0]) for d in descriptors]
print(len(descriptors))
print(descriptors[:10])

29607
['http://id.nlm.nih.gov/mesh/2022/D000327', 'http://id.nlm.nih.gov/mesh/2022/D001639', 'http://id.nlm.nih.gov/mesh/2022/D017680', 'http://id.nlm.nih.gov/mesh/2022/D019290', 'http://id.nlm.nih.gov/mesh/2022/D007652', 'http://id.nlm.nih.gov/mesh/2022/D004744', 'http://id.nlm.nih.gov/mesh/2022/D014267', 'http://id.nlm.nih.gov/mesh/2022/D004288', 'http://id.nlm.nih.gov/mesh/2022/D055312', 'http://id.nlm.nih.gov/mesh/2022/D007073']


In [3]:

def extract_labels(descriptor):
    terms = query(
        """
        SELECT ?l
        WHERE {
        {
            <DESC> meshv:concept ?c .
            ?c meshv:term ?t .
            ?t meshv:prefLabel ?l .
        }
        UNION
        {
            <DESC> rdfs:label ?l .
        }
        UNION
        {
            <DESC> meshv:concept ?c .
            ?c rdfs:label ?l .
        }
        UNION
        {
            <DESC> meshv:term ?t .
            ?t meshv:prefLabel ?l .
        }
        }
        """.replace('DESC', descriptor)
    )

    return set([str(i[0]) for i in terms])

print(extract_labels('http://id.nlm.nih.gov/mesh/2022/D009288'))

{'Naprosyn', 'Sodium Naproxenate', 'Proxen', 'Naproxen Sodium', 'Naproxen', 'Naprosin', 'Synflex', 'Aleve', 'Anaprox'}


In [4]:
related_terms = open('mesh-related-terms.txt', 'w')

for d in descriptors:
    labels = list(extract_labels(d))
    for i in range(len(labels)):
        for j in range(i + 1, len(labels)):
            related_terms.write(labels[i] + '|||' + labels[j] + '|||1.0\n')

related_terms.close()

In [5]:
!wc -l mesh-related-terms.txt

236616 mesh-related-terms.txt
